## Connect to Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import requests
import re

from zipfile import ZipFile
# from keplergl import KeplerGl
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [10]:
day1 = datetime.now()
day1 = day1 - timedelta(days=3)
day1 = day1.strftime("%m%d")

day2 = datetime.now()
day2 = day2 - timedelta(days=2)
day2 = day2.strftime("%m%d")

## Data Imports


In [ ]:
os.chdir('/content/drive/My Drive/safegraph_data/End_Of_Day_Data__Covid19_CSV')
NONRES_DayBefore_df = pd.read_csv(f'{day1}_nonres.csv')
NONRES_DayOf_df = pd.read_csv(f'{day2}_nonres.csv')


RCAC_DayBefore_df = pd.read_csv(f'{day1}_residual.csv')
RCAC_DayOf_df = pd.read_csv(f'{day2}_residual.csv')

EDUC_DayBefore_df = pd.read_csv(f'{day1}_education.csv')
EDUC_DayOf_df = pd.read_csv(f'{day2}_education.csv')


# Pull in ready-to-use 
 - Los Angeles POI (from SafeGraph)
 - Los Angeles POI September (from SafeGraph)
## **Next Step would be to create a separate python file to pull and clean both of these into clean and ready-to-use files for preping the final USC-POI files**

In [ ]:
os.chdir('/content/drive/My Drive/safegraph_data')
la_poi = pd.read_csv('la_poi.csv')
os.chdir('/content/drive/My Drive/safegraph_data/End_Of_Day_Data__Covid19_CSV')
# la_september_month = pd.read_csv('la_september_month.csv') # not really used here

## Functions

In [ ]:
def day_over_day_match(data_input1, data_input2, col_return_lists, match_on, join_type='inner'):
  # Input: 2 dataframes, a list of columns user is interested, the key/list to join by, and the type of join (defaults to inner)
  # Output: dataframe 
  # Function creates matches two daily dataframes given, used to later calculate deltas

  new_df = pd.merge(data_input1, data_input2, on = match_on, how = join_type)[col_return_lists]
  return new_df

In [ ]:
def create_table():
  # Input: None
  # Output: a dataframe
  # Function creates the structure for the final dataframe housing the final data needed

  date_ending_df = pd.DataFrame(columns=['Time Stamp', 'Region', 'Latitude', 'Longitude', 'Number of cases'])
  return date_ending_df

In [ ]:
def get_single_matches(data_input):
  # Input: dataframe
  # Output: dataframe
  # Function returns a dataframe that contains POI that have a single match against Safegraphs data
  
  single_df = data_input[data_input['number_of_candidate_matches']==1]
  return single_df

In [ ]:
def match_with_poi_and_return_df(data_input, safegraph_data_input, return_these_columns, match_on=['safegraph_place_id'], join_type='inner'):
  # Input: dataframe, safegraph dataframe to match with, list of columns wanted, key to join on (defaults to safegraph_place_id), type of join (defaults to inner)
  # Output: dataframe
  # Function merges the given dataframe with the one Safegraph given one

  output_df = pd.merge(data_input, safegraph_data_input, on=match_on, how=join_type)[return_these_columns]
  return output_df

In [ ]:
# single_matches_DayBefore = NONRES_DayBefore_df[NONRES_DayBefore_df['number_of_candidate_matches']==1]
# single_matches_DayOf = NONRES_DayOf_df[NONRES_DayOf_df['number_of_candidate_matches']==1]

# final_matches_DayBefore = pd.merge(single_matches_DayBefore, la_poi, on=['safegraph_place_id'], how='inner')[['customer_location_name','customer_address','customer_total_confirmed_staff','city','region','safegraph_place_id','latitude','longitude','open_hours']]
# final_matches_DayOf = pd.merge(single_matches_DayOf, la_poi, on=['safegraph_place_id'], how='inner')[['customer_location_name','customer_address','customer_total_confirmed_staff','city','region','safegraph_place_id','latitude','longitude','open_hours']]


In [ ]:
today = datetime.now()
today = today - timedelta(days=2)
today = today.strftime("%m-%d-%Y")

## Gather Non-Residential Settings Data

In [ ]:

## Filter to only single matches
single_matches_DayBefore = get_single_matches(NONRES_DayBefore_df)
single_matches_DayOf = get_single_matches(NONRES_DayOf_df)
# single_matches_DayOf.head()


## match with safegraph POI and return that data
return_these_columns = ['customer_location_name','customer_address','customer_total_confirmed_staff','customer_city','customer_state','safegraph_place_id','latitude','longitude','open_hours']

final_matches_DayBefore = match_with_poi_and_return_df(data_input = single_matches_DayBefore, 
                                                  safegraph_data_input = la_poi, 
                                                  return_these_columns = return_these_columns)
final_matches_DayOf = match_with_poi_and_return_df(data_input = single_matches_DayOf, 
                                                  safegraph_data_input = la_poi, 
                                                  return_these_columns = return_these_columns)
# final_matches_DayBefore.head()


day_over_day_columns_to_return_list = ['customer_location_name', 'customer_address', 'customer_total_confirmed_staff_x', 'customer_total_confirmed_staff_y', 'customer_city', 'latitude_x', 'longitude_x']
day_over_dat_match_on_list = ['customer_location_name', 'customer_address', 'customer_city']

## Merge day-over-day data and create/reutrn final day over day delta table
DayBefore_DayOf_delta = day_over_day_match(data_input1 = final_matches_DayBefore, 
                                           data_input2 = final_matches_DayOf, 
                                           col_return_lists = day_over_day_columns_to_return_list, 
                                           match_on = day_over_dat_match_on_list)
DE_DayOf_NONRES = create_table()
# DayBefore_DayOf_delta.head()

DE_DayOf_NONRES['Region'] = DayBefore_DayOf_delta['customer_location_name']
DE_DayOf_NONRES['Latitude'] = DayBefore_DayOf_delta['latitude_x']
DE_DayOf_NONRES['Longitude'] = DayBefore_DayOf_delta['longitude_x']
DE_DayOf_NONRES['Number of cases'] = DayBefore_DayOf_delta['customer_total_confirmed_staff_y'] - DayBefore_DayOf_delta['customer_total_confirmed_staff_x']
DE_DayOf_NONRES['Time Stamp'] = today


# DE_DayOf_NONRES.sort_values('cases', ascending=False).head(5)


# Gather Residential Congregate Settings

In [ ]:

## Filter to only single matches
single_matches_DayBefore = get_single_matches(RCAC_DayBefore_df)
single_matches_DayOf = get_single_matches(RCAC_DayOf_df)
# single_matches_DayOf.head()


# ## match with safegraph POI and return that data
return_these_columns = ['customer_location_name','customer_city_name','customer_number_of_confirmed_staff',
                        'customer_number_of_confirmed_residents','customer_state','safegraph_place_id','latitude',
                        'longitude','open_hours']

final_matches_DayBefore = match_with_poi_and_return_df(data_input = single_matches_DayBefore, 
                                                  safegraph_data_input = la_poi, 
                                                  return_these_columns = return_these_columns)
final_matches_DayOf = match_with_poi_and_return_df(data_input = single_matches_DayOf, 
                                                  safegraph_data_input = la_poi, 
                                                  return_these_columns = return_these_columns)
# final_matches_DayOf.head()

final_matches_DayOf = pd.DataFrame.drop_duplicates(final_matches_DayOf)
final_matches_DayBefore = pd.DataFrame.drop_duplicates(final_matches_DayBefore)


day_over_day_columns_to_return_list = ['customer_location_name', 'customer_city_name', 'customer_number_of_confirmed_staff_x', 'customer_number_of_confirmed_residents_x', 
                                       'customer_number_of_confirmed_staff_y', 'customer_number_of_confirmed_residents_y','latitude_x', 'longitude_x']
day_over_dat_match_on_list = ['customer_location_name', 'customer_city_name', 'safegraph_place_id']

## Merge day-over-day data and create/reutrn final day over day delta table
DayBefore_DayOf_delta = day_over_day_match(data_input1 = final_matches_DayBefore, 
                                           data_input2 = final_matches_DayOf, 
                                           col_return_lists = day_over_day_columns_to_return_list, 
                                           match_on = day_over_dat_match_on_list)

# final_matches_DayOf.head()

## Extra for this section 
DayBefore_DayOf_delta['total_cases_x'] = DayBefore_DayOf_delta['customer_number_of_confirmed_staff_x'] + DayBefore_DayOf_delta['customer_number_of_confirmed_residents_x']
DayBefore_DayOf_delta['total_cases_y'] = DayBefore_DayOf_delta['customer_number_of_confirmed_staff_y'] + DayBefore_DayOf_delta['customer_number_of_confirmed_residents_y']
DayBefore_DayOf_delta = DayBefore_DayOf_delta[['customer_location_name', 'customer_city_name', 'total_cases_x', 'total_cases_y', 'latitude_x', 'longitude_x']]

# DayBefore_DayOf_delta.head()

DE_DayOf_RCAC = create_table()


DE_DayOf_RCAC['Region'] = DayBefore_DayOf_delta['customer_location_name']
DE_DayOf_RCAC['Latitude'] = DayBefore_DayOf_delta['latitude_x']
DE_DayOf_RCAC['Longitude'] = DayBefore_DayOf_delta['longitude_x']
DE_DayOf_RCAC['Number of cases'] = DayBefore_DayOf_delta['total_cases_y'] - DayBefore_DayOf_delta['total_cases_x']
DE_DayOf_RCAC['Time Stamp'] = today


# pd.DataFrame.drop_duplicates(DE_DayOf_RCAC)
# DE_DayOf_RCAC.sort_values('cases', ascending=False).head()


# Gather Educational Settings

In [ ]:

## Filter to only single matches
single_matches_DayBefore = get_single_matches(EDUC_DayBefore_df)
single_matches_DayOf = get_single_matches(EDUC_DayOf_df)
# single_matches_DayOf.head()


## match with safegraph POI and return that data
return_these_columns = ['customer_location_name','customer_address','customer_total_confirmed_staff', 'customer_total_confirmed_students','customer_city','customer_state','safegraph_place_id','latitude','longitude','open_hours']

final_matches_DayBefore = match_with_poi_and_return_df(data_input = single_matches_DayBefore, 
                                                  safegraph_data_input = la_poi, 
                                                  return_these_columns = return_these_columns)
final_matches_DayOf = match_with_poi_and_return_df(data_input = single_matches_DayOf, 
                                                  safegraph_data_input = la_poi, 
                                                  return_these_columns = return_these_columns)
# single_matches_DayBefore


day_over_day_columns_to_return_list = ['customer_location_name', 'customer_address', 'customer_total_confirmed_staff_x', 'customer_total_confirmed_students_x', 
                                       'customer_total_confirmed_staff_y', 'customer_total_confirmed_students_y', 'customer_city', 'latitude_x', 'longitude_x']
day_over_dat_match_on_list = ['customer_location_name', 'customer_address', 'customer_city']

## Merge day-over-day data and create/reutrn final day over day delta table
DayBefore_DayOf_delta = day_over_day_match(data_input1 = final_matches_DayBefore, 
                                           data_input2 = final_matches_DayOf, 
                                           col_return_lists = day_over_day_columns_to_return_list, 
                                           match_on = day_over_dat_match_on_list)

## Extra for this section 
DayBefore_DayOf_delta['total_cases_x'] = DayBefore_DayOf_delta['customer_total_confirmed_staff_x'] + DayBefore_DayOf_delta['customer_total_confirmed_students_x']
DayBefore_DayOf_delta['total_cases_y'] = DayBefore_DayOf_delta['customer_total_confirmed_staff_y'] + DayBefore_DayOf_delta['customer_total_confirmed_students_y']
DayBefore_DayOf_delta = DayBefore_DayOf_delta[['customer_location_name', 'customer_city', 'total_cases_x', 'total_cases_y', 'latitude_x', 'longitude_x']]

DE_DayOf_EDUC = create_table()
# DayBefore_DayOf_delta.head()

DE_DayOf_EDUC['Region'] = DayBefore_DayOf_delta['customer_location_name']
DE_DayOf_EDUC['Latitude'] = DayBefore_DayOf_delta['latitude_x']
DE_DayOf_EDUC['Longitude'] = DayBefore_DayOf_delta['longitude_x']
DE_DayOf_EDUC['Number of cases'] = DayBefore_DayOf_delta['total_cases_y'] - DayBefore_DayOf_delta['total_cases_y']
DE_DayOf_EDUC['Time Stamp'] = today


# DE_DayOf_EDUC.sort_values('cases', ascending=False)


## Combine these 3 sections into 1

In [ ]:
ending_day_delta = pd.concat([DE_DayOf_EDUC, DE_DayOf_RCAC, DE_DayOf_NONRES])

In [ ]:
today_format = datetime.now()
today_format = today_format - timedelta(days=2)
today_format = today_format.strftime("%b%d_%Y")

## Export Data

In [ ]:
ending_day_delta.sort_values('Number of cases', ascending=False).to_csv(f'/content/drive/My Drive/safegraph_data/End_Of_Day_Data__Covid19_CSV/Ending_{today_format}_day_delta.csv', index = False)
# ending_day_delta.sort_values('Number of cases', ascending=False).head(18)
# ending_day_delta.head()

In [ ]:
# # !pwd
# os.chdir('/content/drive/My Drive/safegraph_data/SINGLE_POI_SAMPLE/')
# ending_day_delta[ending_day_delta['Region']=='Vallarta Supermarket #25'].to_csv(f'/content/drive/My Drive/safegraph_data/SINGLE_POI_SAMPLE/Covid-19-single-poi_{today_format}.csv', index = False)